In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# Setup feedback system
from learntools.core import binder
binder.bind(globals())

# Setup notebook
from pathlib import Path
from learntools.time_series.style import *  # plot style settings
from learntools.time_series.utils import plot_periodogram, seasonal_plot

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import calendar
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder

from statsmodels.tsa.deterministic import CalendarFourier, CalendarSeasonality, CalendarTimeTrend, DeterministicProcess
from statsmodels.graphics.tsaplots import plot_pacf

from sklearn.metrics import mean_squared_log_error
from sklearn.metrics import mean_squared_error

In [3]:
comp_dir = Path('../input/store-sales-time-series-forecasting')

In [4]:
sdate = '2015-08-15'
edate = '2017-08-15'

In [5]:
train_data = pd.read_csv(comp_dir / 'train.csv')
train_data.head()

In [6]:
store_sales = pd.read_csv(comp_dir / 'train.csv',
                       usecols=['date', 'store_nbr', 'family', 'sales','onpromotion'],
                       dtype={'store_nbr': 'category', 'family': 'category', 'sales': 'float32', 'onpromotion': 'int32'},
                       parse_dates=['date'],
                       infer_datetime_format=True
                      )
store_sales['date'] = store_sales.date.dt.to_period('D')

m_index = pd.MultiIndex.from_product([store_sales["store_nbr"].unique(),
                                      store_sales["family"].unique(),
                                      pd.date_range(start="2013-1-1", end="2017-8-15", freq="D").to_period('D')] # to get missing Christmas Days
                                     ,names=["store_nbr","family", "date"])
store_sales = store_sales.set_index(["store_nbr","family", "date"]).reindex(m_index, fill_value=0).sort_index()

store_sales = store_sales.unstack(['store_nbr', 'family']).fillna(0) # there are lots!
store_sales = store_sales.stack(['store_nbr', 'family'])
store_sales = store_sales[['sales','onpromotion']] # reorder columns to be in the expected order

store_sales

In [7]:
CALENDAR = pd.DataFrame(pd.period_range(start='2013-01-01', end = '2017-08-31'),columns=['date'])
CALENDAR = CALENDAR.set_index(['date']).sort_index()

CALENDAR

In [8]:
## oil data

oil_data = pd.read_csv(comp_dir / 'oil.csv',
                      parse_dates =  ['date'],
                      infer_datetime_format = True,
                      dtype = {'dcoilwtico' : 'float32'}
                      )
oil_data['date'] = oil_data.date.dt.to_period('D')
oil_data = oil_data.set_index(['date']).sort_index()

oil_df = oil_data.copy()
oil_df

In [9]:
oil_df['dcoilwtico']  = oil_df['dcoilwtico'].interpolate(limit_direction='both')
oil_df["oil_ma"] = oil_df['dcoilwtico'].rolling(window=7).mean()
oil_df["oil_sd"] = oil_df['dcoilwtico'].rolling(window=7).std()

oil_df

In [10]:
CALENDAR = CALENDAR.join(oil_df, on = 'date')
CALENDAR.ffill(inplace=True)
CALENDAR.bfill(inplace=True)

CALENDAR.plot(figsize=(20,10), title= "Moving Average oil", xlabel= "date", rot=90)

In [11]:
CALENDAR['dofw'] = CALENDAR.index.dayofweek
CALENDAR['wage_day'] = (CALENDAR.index.day == CALENDAR.index.daysinmonth) | (CALENDAR.index.day == 15).astype(int) 

CALENDAR

#### Holiday dataset analysis

In [12]:
## Checking the Hoiliday dataset..

holiday_data = pd.read_csv(comp_dir / 'holidays_events.csv',
                           dtype={
                                  'type': 'category',
                                  'locale': 'category',
                                  'locale_name': 'category',
                                  'description': 'category',
                                  'transferred': 'bool'
                                 },
                          parse_dates=['date'],
                          infer_datetime_format=True)

holiday_data = holiday_data.set_index('date').to_period('D')

holiday_data.head()

In [13]:
holiday_df = holiday_data.query("locale in ['National', 'Regional']")

## Remove duplicates from the 'holiday' dataset
holiday_df = holiday_df.groupby(holiday_df.index).first()

print(holiday_df.shape)
holiday_df

In [14]:
CALENDAR['WD'] = 1
CALENDAR.loc[CALENDAR.dofw > 4, 'WD'] = 0

CALENDAR = CALENDAR.merge(holiday_df, on='date', how='left')


CALENDAR.loc[CALENDAR.type == 'Bridge'  , 'WD'] = 0
CALENDAR.loc[CALENDAR.type == 'Work Day', 'WD'] = 1
CALENDAR.loc[CALENDAR.type == 'Transfer', 'WD'] = 0
CALENDAR.loc[(CALENDAR.type == 'Holiday') & (CALENDAR.transferred == False), 'WD'] = 0
CALENDAR.loc[(CALENDAR.type == 'Holiday') & (CALENDAR.transferred == True ), 'WD'] = 1

CALENDAR = pd.get_dummies(CALENDAR, columns = ['dofw'], drop_first = True) 
CALENDAR = pd.get_dummies(CALENDAR, columns = ['type'])

CALENDAR.drop(['locale', 'locale_name', 'description', 'transferred'], axis=1, inplace=True)

CALENDAR

In [15]:
CALENDAR.columns

In [16]:
keep_cols = ['oil_ma', 'oil_sd', 'wage_day', 'WD', 'dofw_1', 'dofw_2', 'dofw_3', 'dofw_4', 'dofw_5',
             'dofw_6', 'type_Additional', 'type_Bridge', 'type_Event', 'type_Holiday', 'type_Transfer', 'type_Work Day'
             ]

CALENDAR = CALENDAR[keep_cols]
CALENDAR

In [17]:
CALENDAR_train = CALENDAR.loc[sdate:edate]

X_time = CALENDAR_train.copy()

X_time

In [18]:
y = store_sales.unstack(['store_nbr', 'family']).loc[sdate:edate,'sales']
y

In [19]:
index = y.index
tt = CalendarTimeTrend("D", True, order=2)
four_M = CalendarFourier("M", 12)
four_A = CalendarFourier("A", 4)
seas = CalendarSeasonality("D", "W")
det_proc = DeterministicProcess(index, additional_terms=[tt, seas, four_M, four_A], drop=True)
X_dp = det_proc.in_sample()

X_dp

In [20]:
X1 = pd.concat([X_dp, X_time], axis=1).fillna(0.0)
X1

In [21]:
def encode_categoricals(df, columns):
    le = LabelEncoder()  # from sklearn.preprocessing
    for col in columns:
        df[col] = le.fit_transform(df[col])
    return df

In [22]:
def make_X2_lags(ts, lags, lead_time=1, name='y', stack_cols=None):
    ts = ts.unstack(stack_cols)
    df = pd.concat(
        {
            f'{name}_lag_{i}': ts.shift(i, freq="D") # freq adds i extra day(s) to end: only one extra day is needed so rest will be dropped
            for i in range(lead_time, lags + lead_time)
        },
        axis=1)
    df = df.stack(stack_cols).reset_index()
    df = encode_categoricals(df, stack_cols)
    df = df.set_index('date').sort_values(by=stack_cols) # return sorted so can correctly compute rolling means (if desired)
    return df

In [23]:
def make_X2_features(df, y_resid):
    stack_columns = ['store_nbr', 'family']
    
    # promo_lag features
    shifted_promo_df = make_X2_lags(df.squeeze(), lags=2, name='promo', stack_cols=['store_nbr', 'family'])
    
    #y_lag features
    shifted_y_df = make_X2_lags(y_resid, lags=2, name='y_res', stack_cols=stack_columns)

    df = df.reset_index(stack_columns)
    X2 = encode_categoricals(df, stack_columns)
    
    X2["day_of_w"] = X2.index.dayofweek    
    
    X2['wage_day'] = (X2.index.day == X2.index.daysinmonth) | (X2.index.day == 15).astype(int)
    
    X2 = X2.merge(shifted_promo_df, on=['date', 'store_nbr', 'family'], how='left')
    X2 = X2.merge(shifted_y_df, on=['date', 'store_nbr', 'family'], how='left')    
    X2 = X2.fillna(0.0)
    
    
    return X2

In [24]:
class BoostedHybrid:
    def __init__(self, model_1, model_2):
        self.model_1 = model_1
        self.model_2 = model_2
        self.y_columns = None
        self.stack_cols = None
        self.y_resid = None 
        
    def fit1(self, X_1, y, stack_cols=None):                  
        self.model_1.fit(X_1, y)
        y_fit = pd.DataFrame(
            self.model_1.predict(X_1),
            index=X_1.index, columns=y.columns,
        )
        self.y_resid = y - y_fit
        self.y_resid_stacked = self.y_resid.stack(stack_cols).squeeze() # wide to long
        self.y_columns = y.columns # Save for predict method
        
        
    def fit2(self, X_2, stack_cols=None):
        self.model_2.fit(X_2, self.y_resid_stacked) # Train model_2
        self.stack_cols = stack_cols # Save for predict method
        
    
    def predict(self, X_1, X_2):
        y_pred = pd.DataFrame(
            self.model_1.predict(X_1),
            index=X_1.index,
            columns=self.y_columns,
        )
        y_pred = y_pred.stack(self.stack_cols).squeeze()  # wide to long
        y_pred += self.model_2.predict(X_2) # Add model_2 predictions to model_1 predictions
        return y_pred.unstack(self.stack_cols)        

In [25]:
store_sales_X2 = store_sales.unstack(['store_nbr', 'family']).loc[sdate:edate]
store_sales_X2

In [26]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import ElasticNet


model = BoostedHybrid(model_1=ElasticNet(alpha=0.5, fit_intercept=False, random_state=42), model_2=RandomForestRegressor())

In [27]:
model.fit1(X1, y, stack_cols=['store_nbr', 'family'])

In [28]:
ax = y.unstack(level=0).plot(**plot_params, figsize=(16, 4))
ax.set_title('Sales - Actual');

In [29]:
ax = model.y_resid.unstack(level=0).plot(**plot_params, figsize=(16, 4))
ax.set_title('Sales - Deseasonalized');

In [30]:
X2 = make_X2_features(store_sales_X2 # preparing X2 for hybrid model 2
                       .drop('sales', axis=1)
                       .stack(['store_nbr', 'family']),
                       model.y_resid_stacked)

X2

In [31]:
model.fit2(X2, stack_cols=['store_nbr', 'family'])

In [32]:
y_pred = model.predict(X1, X2).clip(0.0)

In [33]:
print("Model RMSE : ", mean_squared_log_error(y, y_pred)**0.5)

In [34]:
## plotting our predictions over Actual sales

families = ['AUTOMOTIVE','BEAUTY','BEVERAGES','GROCERY I', "BOOKS", "BABY CARE", "CELEBRATION"]
store_nbr = '1'

fig, axs = plt.subplots(len(families), figsize=(25,50))
for i in range(len(families)):
    axs[i] = y.loc(axis=1)[store_nbr, families[i]].loc[sdate:].plot(ax=axs[i], label='Sales')
    axs[i] = y_pred.loc(axis=1)[store_nbr, families[i]].plot(ax=axs[i], label='Model_Pred')
        
    axs[i].set_title(f'{families[i]} sales at store : {store_nbr}');
    axs[i].legend();

In [35]:
# Results of the training stage with RF

y_pred_train   = y_pred.stack(['store_nbr', 'family']).reset_index()
y_target = y.stack(['store_nbr', 'family']).reset_index().copy()
y_target.rename(columns={y_target.columns[3]:'sales'}, inplace=True)

y_target['sales_pred'] = y_pred_train[0].clip(0.) # Sales should be >= 0
y_target['store_nbr'] = y_target['store_nbr'].astype(int)

print('\nRMSLE by family\n')
display(y_target.groupby('family').apply(lambda r: mean_squared_log_error(r['sales'], r['sales_pred'])))

print('\nRMSLE by StoreNumber\n')
display(y_target.sort_values(by="store_nbr").groupby('store_nbr').apply(lambda r: mean_squared_log_error(r['sales'], r['sales_pred'])))


## Test data

In [36]:
test_data = pd.read_csv(comp_dir / 'test.csv')
test_data

In [37]:
df_test = pd.read_csv(
                    comp_dir / 'test.csv',
                    dtype={
                        'store_nbr': 'category',
                        'family': 'category',
                        'onpromotion': 'uint32',
                         },
        parse_dates=['date'],
       infer_datetime_format=True,
)
df_test['date'] = df_test.date.dt.to_period('D')
df_test = df_test.set_index(['date','store_nbr', 'family']).sort_index()

df_test

In [38]:
test_df = df_test.drop('id', axis=1)
test_df

In [39]:
start_test = '2017-08-16'
end_test = '2017-08-31'

In [40]:
CALENDAR_test = CALENDAR.loc[start_test:end_test]

X_test_time = CALENDAR_test.copy()

X_test_time

In [41]:
X_test_dp = det_proc.out_of_sample(steps=16)

X_test_dp

In [42]:
X1_test = pd.concat([X_test_dp, X_test_time], axis=1).fillna(0.0)
X1_test

In [43]:
X2_test = make_X2_features(test_df, model.y_resid_stacked)  # preparing X2 for hybrid model 2                           

X2_test

In [44]:
test_pred = pd.DataFrame(model.predict(X1_test, X2_test).clip(0.0), index=X1_test.index, columns=y.columns)

test_pred = test_pred.stack(['store_nbr', 'family'])

test_pred


In [45]:
df_submit = pd.read_csv(comp_dir / 'sample_submission.csv', index_col='id')
df_submit.sales = test_pred.values
df_submit.to_csv('submission.csv', index=True)